# 03 Evaluate fastText

## TODO

- auto split train and valid
- auto epoch
- print samples (DONE)

In [1]:
import fasttext

from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [2]:
# Hjælpefunktion til at omregne til procent
def to_perc(a, b):
    return a / (a+b) * 100

In [41]:
# Funktion til at returnére den mest optimale epoch iteration af datasættet
def model_trainer():
    models_train = []
    models_valid = []
    peak_epoch = 0
    peak_value = 0.0

    with tqdm(total=epoch+1) as progress:
        for i in range(1, epoch+1):
            progress.update(1)

            model = fasttext.train_supervised(input=data_train, epoch=i, lr=lr, wordNgrams=wordNgrams, bucket=bucket, dim=dim, loss=loss)

            models_train.append(model.test(data_train, k=top_labels))
            models_valid.append(model.test(data_valid, k=top_labels))
            
            if (len(models_train) == 1):
                print('             Training  |    Validation')
                print('        ', '{0:6}'.format(models_train[i-1][0]), '{:.1f}'.format(to_perc(models_train[i-1][0], models_valid[i-1][0])) + '%',
                      ' | ', '{0:6}'.format(models_valid[i-1][0]), '{:.1f}'.format(to_perc(models_valid[i-1][0], models_train[i-1][0])) + '%')
                print('                       |')
                print('Epoch   ', 'Prec.', 'Recall  | ', 'Prec.', 'Recall')
                print('-----------------------+---------------------------')
            
            if (models_valid[i-1][1] > peak_value):
                peak_value = models_valid[i-1][1]
                peak_epoch = i
                precision = '\033[92mHigher\033[0m'
            else:
                precision = '\033[91mLower\033[0m'

            print('{0:2}'.format(i), ' ==> ', '{:.3f}'.format(models_train[i-1][1]), ' {:.3f}'.format(models_train[i-1][2]),
                  ' |', ' {:.3f}'.format(models_valid[i-1][1]), ' {:.3f}'.format(models_valid[i-1][2]), ' ==> ', precision)

        print('\n    -->  Epoch peaked at', '\033[1m' + '\033[94m' + str(peak_epoch))

        model = fasttext.train_supervised(input=data_train, epoch=peak_epoch, lr=lr, wordNgrams=wordNgrams, bucket=bucket, dim=dim, loss=loss)
        model.save_model('./data/fasttext_trained_model.bin')
        
        return peak_epoch, models_train, models_valid

In [18]:
# Visualisér modellen via plotting
def model_plot(title, i, model_valid_epoch, models_train, models_valid):
    threshold = 0.2
    y_range_min = min([y[i] for y in models_valid if y[i] > threshold])
    y_range_max = max([y[i] for y in models_train if y[i] > threshold])
    
    plt.plot([pr[i] for pr in models_train], label='Training')
    plt.plot([pr[i] for pr in models_valid], label='Validation')
    plt.axvline(model_valid_epoch, linewidth=2, color='g', linestyle='--', label=('Epoch Peak'))

    plt.title('Model (' + title + ')')
    plt.ylabel(title)
    plt.xlabel('Epoch')
    plt.legend(loc='upper left')
    plt.ylim([y_range_min, y_range_max+0.01])
    
    return plt.show()

In [ ]:
# Træn modellen ud fra givne parameter
data_train = './data/test.train'
data_valid = './data/test.valid'
epoch      = 30
lr         = 0.2
wordNgrams = 2
bucket     = 20000
dim        = 100
loss       = 'ova'
top_labels = 2

mt = model_trainer()
model_plot('Precision', 1, mt[0], mt[1], mt[2])
model_plot('Recall', 2, mt[0], mt[1], mt[2])

             Training  |    Validation
           4998 69.2%  |    2229 30.8%
                       |
Epoch    Prec. Recall  |  Prec. Recall
-----------------------+---------------------------
 1  ==>  0.433  0.865  |  0.424  0.848  ==>  Higher
 2  ==>  0.446  0.892  |  0.436  0.873  ==>  Higher
 3  ==>  0.450  0.900  |  0.441  0.881  ==>  Higher
 4  ==>  0.453  0.906  |  0.444  0.889  ==>  Higher
 5  ==>  0.454  0.907  |  0.447  0.894  ==>  Higher
 6  ==>  0.454  0.907  |  0.448  0.897  ==>  Higher
 7  ==>  0.455  0.909  |  0.450  0.900  ==>  Higher
 8  ==>  0.456  0.913  |  0.451  0.903  ==>  Higher
 9  ==>  0.459  0.919  |  0.453  0.905  ==>  Higher
10  ==>  0.464  0.928  |  0.456  0.913  ==>  Higher
